In [1]:
from astropy.io import ascii, fits
import astropy
import pylab as plt
%matplotlib inline
from astropy import wcs
from astropy.table import Table,Column,join,hstack
from astropy.coordinates import SkyCoord
from astropy import units as u
import pymoc
import glob
from time import sleep
import os


import numpy as np
import xidplus
from xidplus import moc_routines
import pickle
import xidplus.catalogue as cat

import sys
from herschelhelp_internal.utils import inMoc,flux_to_mag
from xidplus.stan_fit import SPIRE

import aplpy
import seaborn as sns
#sns.set(color_codes=True)
import pandas as pd
#sns.set_style("white")
import xidplus.posterior_maps as postmaps
import pyvo as vo



First we select the field that the sources we are considering are in. If the sources span multiple fields that each field will need to be run individually as the FIR maps from seperate fields cannot be easily combined.

In [3]:
fields = ['AKARI-NEP',
 'AKARI-SEP',
 'Bootes',
 'CDFS-SWIRE',
 'COSMOS',
 'EGS',
 'ELAIS-N1',
 'ELAIS-N2',
 'ELAIS-S1',
 'GAMA-09',
 'GAMA-12',
 'GAMA-15',
 'HDF-N',
 'Herschel-Stripe-82',
 'Lockman-SWIRE',
 'NGP',
 'SA13',
 'SGP',
 'SPIRE-NEP',
 'SSDF',
 'XMM-13hr',
 'XMM-LSS',
 'xFLS']

field_use = fields[6]
print(field_use)

ELAIS-N1


Here you provide the coordinate of the objects you are planning to run XID+ on and their ID's if any 
If no ids are provided then they will be numbered 1-N)

In [19]:
ras = [242,243]#enter your ra here as a list of numpy array
decs = [55,55] #enter your dec here as a list or numpy array
object_coords = SkyCoord(ra=ras*u.degree,dec=decs*u.degree)

ids = [] #add your ids here as a list or numpy array
if len(ids)==0:
    ids = np.arange(0,len(ras),1)

Run the pyvo query to create a table of all help sources within the desired radius of your objects

In [12]:
#setup a connection to the HELP VO server at Sussex
search_radius = 60/3600 #distance away from object that the VO query will look for galaxies in degrees
#for SPIRE AND PACS we recommend 60" and for MIPS we reccomend 30"


service = vo.dal.TAPService("https://herschel-vos.phys.sussex.ac.uk/__system__/tap/run/tap")

for n,coords in enumerate(object_coords):
    ra = coords.ra
    dec = coords.dec
    query_spire_pacs = """
        SELECT ra, dec, help_id, flag_optnir_det, f_mips_24
        FROM herschelhelp.main
        WHERE (
        herschelhelp.main.field = '{}' AND
        herschelhelp.main.falg_optnir_det>=5 AND
        herschelhelp.main.f_mips_24>20
        ) AND
        WHERE CONTAINS(POINT('ICRS',ra, dec), CIRCLE('ICRS',{},{},{}))=1
        """.format(field_use,ra,dec,search_radius)
    
    query_mips = """
        SELECT ra, dec, help_id, flag_optnir_det, f_irac_i1, f_irac_i2, f_irac_i3, f_irac_i4
        FROM herschelhelp.main
        WHERE (
        herschelhelp.main.field = '{}' AND
        herschelhelp.main.falg_optnir_det>=5 AND
        ) AND
        WHERE CONTAINS(POINT('ICRS',ra, dec), CIRCLE('ICRS',{},{},{}))=1
        """.format(field_use,ra,dec,search_radius)
        
        
    try:
        job = service.submit_job(query)
        job.run()

        while job.phase == "EXECUTING":
            print("Job running")
            sleep(5)
        print('Job finsihed')        

        if n==0:
            prior_help = job.fetch_result().to_table()
            print('table created with {} rows'.format(len(table)))
        else:
            result = job.fetch_result().to_table()
            prior_help = astropy.table.vstack([result,table],join_type='outer')
            print('table editied, added {} rows'.format(len(result)))


        done_fields.append(field)
    except:
        print('VO call failed')
    job.delete()

Job running


job finsihed
VO call failed


In [18]:
print(len(prior_help))
prior_help[:5]

NameError: name 'prior_cat' is not defined

Run the below cell if you are running XID+ on SPIRE or PACS maps

In [ ]:
cra = Column(ras,name='ra')
cdec = Column(decs,name='dec')
cids = Column(ids,name='help_id')
cdet = Column(np.zeros(len(ras))-99,name='flag_optnir_det')
cmips = Column(np.zeros(len(ras))*np.nan,name='f_mips_24')
prior_new = Table()
prior_new.add_columns([cra,cdec,cids,cdet,cmips])


prior_cat = vstack([prior_help,prior_new])
len(prior_cat)
prior_cat[:5]

Run the below cells if you are running XID+ on MIPS maps

In [ ]:
#provides limits on teh flat prior used in XID based on the galaxies IRAC fluxes
MIPS_lower=np.full(len(prior_help),0.0)
MIPS_upper=np.full(len(prior_help),1E5)
for i in range(len(prior_cat)):
    if np.isnan(prior_cat['f_irac_i4'][i])==False:
        MIPS_lower[i]=prior_cat['f_irac_i4'][i]/500.0
        MIPS_upper[i]=prior_cat['f_irac_i4'][i]*500.0
    elif np.isnan(prior_cat['f_irac_i3'][i])==False:
        MIPS_lower[i]=prior_cat['f_irac_i3'][i]/500.0
        MIPS_upper[i]=prior_cat['f_irac_i3'][i]*500.0
    elif np.isnan(prior_cat['f_irac_i2'][i])==False:
        MIPS_lower[i]=prior_cat['f_irac_i2'][i]/500.0
        MIPS_upper[i]=prior_cat['f_irac_i2'][i]*500.0
    elif np.isnan(prior_cat['f_irac_i1'][i])==False:
        MIPS_lower[i]=prior_cat['f_irac_i1'][i]/500.0
        MIPS_upper[i]=prior_cat['f_irac_i1'][i]*500.0
        
mips_lower_col = Column(MIPS_lower,name='MIPS_lower')
mips_upper_col = Column(MIPS_upper,name='MIPS_upper')
prior_help.add_columns([mips_lower_col,mips_upper_col])

In [ ]:
#add your IRAC fluxes here, if your objects don't have IRAC fluxes then they will be set to nan
i1_f = np.zeros(len(ras))*np.nan
i2_f = np.zeros(len(ras))*np.nan
i3_f = np.zeros(len(ras))*np.nan
i4_f = np.zeros(len(ras))*np.nan

cra = Column(ras,name='ra')
cdec = Column(decs,name='dec')
cids = Column(ids,name='help_id')
cdet = Column(np.zeros(len(ras))-99,name='flag_optnir_det')
ci1 = Column(i1_f,name='f_irac_i1')
ci2 = Column(i2_f,name='f_irac_i2')
ci3 = Column(i3_f,name='f_irac_i3')
ci4 = Column(i4_f,name='f_irac_i4')



MIPS_lower=np.full(len(lofar_prior),0.0)
MIPS_upper=np.full(len(lofar_prior),1E5)
for i in range(len(lofar_prior)):
    if np.isnan(lofar_prior['f_irac_i4'][i])==False:
        MIPS_lower[i]=lofar_prior['f_irac_i4'][i]/500.0
        MIPS_upper[i]=lofar_prior['f_irac_i4'][i]*500.0
    elif np.isnan(lofar_prior['f_irac_i3'][i])==False:
        MIPS_lower[i]=lofar_prior['f_irac_i3'][i]/500.0
        MIPS_upper[i]=lofar_prior['f_irac_i3'][i]*500.0
    elif np.isnan(lofar_prior['f_irac_i2'][i])==False:
        MIPS_lower[i]=lofar_prior['f_irac_i2'][i]/500.0
        MIPS_upper[i]=lofar_prior['f_irac_i2'][i]*500.0
    elif np.isnan(lofar_prior['f_irac_i1'][i])==False:
        MIPS_lower[i]=lofar_prior['f_irac_i1'][i]/500.0
        MIPS_upper[i]=lofar_prior['f_irac_i1'][i]*500.0
        
mips_lower_col = Column(MIPS_lower,name='MIPS_lower')
mips_upper_col = Column(MIPS_upper,name='MIPS_upper')
prior_new = Table()
prior_new.add_columns([cra,cdec,cids,cdet,ci1,ci2,ci3,ci4,mips_lower_col,mips_upper_col])


prior_cat = vstack([prior_help,prior_new])
len(prior_cat)

Now that we have created the prior we can run XID+

## Load in the FIR maps

here we load in the SPIRE maps but you can substitue this with PACS and MIPS yourself

In [ ]:
#Read in the herschel images
imfolder='../../../../../HELP/dmu_products/dmu19/dmu19_HELP-SPIRE-maps/data/'

pswfits=imfolder+'ELAIS-N1_SPIRE250_v1.0.fits'#SPIRE 250 map
pmwfits=imfolder+'ELAIS-N1_SPIRE350_v1.0.fits'#SPIRE 350 map
plwfits=imfolder+'ELAIS-N1_SPIRE500_v1.0.fits'#SPIRE 500 map

#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist['image'].header

im250=hdulist['image'].data*1.0E3 #convert to mJy
nim250=hdulist['error'].data*1.0E3 #convert to mJy
w_250 = wcs.WCS(hdulist['image'].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist['image'].header

im350=hdulist['image'].data*1.0E3 #convert to mJy
nim350=hdulist['error'].data*1.0E3 #convert to mJy
w_350 = wcs.WCS(hdulist['image'].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist['image'].header 
im500=hdulist['image'].data*1.0E3 #convert to mJy
nim500=hdulist['error'].data*1.0E3 #convert to mJy
w_500 = wcs.WCS(hdulist['image'].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

Create a moc around each of your objects that will be used to cut doen the SPIRE image

In [ ]:
moc=pymoc.util.catalog.catalog_to_moc(object_coords,search_radius,15)

finish initalising the prior 

In [ ]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu, moc=moc)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(prior_cat['ra'],prior_cat['dec'],'prior_cat',ID=prior_cat['help_id'])#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Gaussian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu, moc=moc)
prior350.prior_cat(prior_cat['ra'],prior_cat['dec'],'prior_cat',ID=prior_cat['help_id'])
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu, moc=moc)
prior500.prior_cat(prior_cat['ra'],prior_cat['dec'],'prior_cat',ID=prior_cat['help_id'])
prior500.prior_bkg(-5.0,5)

In [ ]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires PRF as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [ ]:
print('fitting '+ str(prior250.nsrc)+' sources \n')
print('using ' +  str(prior250.snpix)+', '+ str(prior350.snpix)+' and '+ str(prior500.snpix)+' pixels')

In [ ]:
prior250.get_pointing_matrix()
prior350.get_pointing_matrix()
prior500.get_pointing_matrix()

In [ ]:
prior250.upper_lim_map()
prior350.upper_lim_map()
prior500.upper_lim_map()

run XID+ and save the output

In [ ]:
from xidplus.stan_fit import SPIRE
fit=SPIRE.all_bands(prior250,prior350,prior500,iter=1000)

In [ ]:
posterior=xidplus.posterior_stan(fit,[prior250,prior350,prior500])
xidplus.save([prior250,prior350,prior500],posterior,'YOUR_FILE_NAME_HERE')